<a href="https://colab.research.google.com/github/MarkusKhoa/VLSP2020-ReINTEL/blob/main/Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install sentencepiece
!pip install emoji
!pip install vncorenlp
!pip install sentence_transformers

In [2]:
import torch
import emoji
import re
import gc
import os
import json
import csv
import logging as lg
import pandas as pd
import numpy as np
import math

from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
from sklearn.model_selection import StratifiedKFold, KFold
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from torch import nn
from torch.nn import LSTM
from transformers import *
from vncorenlp import VnCoreNLP
from nltk.tokenize import TweetTokenizer
from pandas import DataFrame

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_path = "/content/drive/MyDrive/VLSP-Fake-News-Detection/public_train.csv"
val_path = "/content/drive/MyDrive/VLSP-Fake-News-Detection/val.csv"
test_path = "/content/drive/MyDrive/VLSP-Fake-News-Detection/final_private_test.csv"

In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [6]:
class ElectraReINTELClassification(ElectraPreTrainedModel):
    def __init__(self, config):
        super(ElectraReINTELClassification, self).__init__(config=config)
        self.electra = ElectraModel(config)
        self.num_labels = config.num_labels
        self.init_weights()
        self.ln = torch.nn.Linear(config.hidden_size * 4, self.num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                start_positions=None, end_positions=None):
        outputs = self.electra(input_ids, attention_mask=attention_mask, position_ids=position_ids, head_mask=head_mask)[1]
        cls_output = torch.cat((outputs[-1][:, 0, ...], outputs[-2][:, 0, ...], outputs[-3][:, 0, ...], outputs[-4][:, 0, ...]), -1)
        logits = self.ln(cls_output)
        return logits

In [7]:
class RobertaReINTELClassification(BertPreTrainedModel):
    def __init__(self, config):
        super(RobertaReINTELClassification, self).__init__(config)
        self.roberta = RobertaModel(config)
        self.num_labels = config.num_labels
        self.outputs = torch.nn.Linear(config.hidden_size * 4, self.num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                start_positions=None, end_positions=None):
        outputs = self.roberta(input_ids, attention_mask=attention_mask)
        cls_output = torch.cat((outputs[2][-1][:, 0, ...], outputs[2][-2][:, 0, ...], outputs[2][-3][:, 0, ...], outputs[2][-4][:, 0, ...]), -1)
        logits = self.outputs(cls_output)
        return logits

In [8]:
class BertReINTELClassification(BertPreTrainedModel):
    def __init__(self, config):
        super(BertReINTELClassification, self).__init__(config=config)
        self.bert = BertModel(config)
        self.num_labels = config.num_labels
        self.ln = torch.nn.Linear(config.hidden_size * 4, self.num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                start_positions=None, end_positions=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, position_ids=position_ids, head_mask=head_mask)[2]

        cls_output = torch.cat((outputs[-1][:, 0, ...], outputs[-2][:, 0, ...], outputs[-3][:, 0, ...], outputs[-4][:, 0, ...]), -1)
        logits = self.ln(cls_output)
        return logits

In [9]:
def normalizeToken(token):
    if len(token) == 1:
        return emoji.demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

In [10]:
def isnan(s):
    return s != s

# Tiền xử lý text
def normalizePost(post, tweet_tokenizer, vncorenlp, use_segment=False, remove_punc_stopword=False, lowercase_opt=False, truncation_method="head_only", length=512):
    post = post.strip()
    URL_pattern = r"(?:http?s://|www.)[^\"]+"
    hashtag_pattern = r"#\w+"

    post = re.sub(URL_pattern, "", post)
    post = re.sub(hashtag_pattern, "hashtag", post)
    post = re.sub("\.+",".", post)
    #post = re.sub("#\s+", " ", post)
    post = re.sub("\*+", " ", post)
    post = re.sub("\$+", "đô ", post)
    post = re.sub("-{2,}", "", post)
    post = re.sub("\@+", "", post)
    post = re.sub("\[[0-9]?[0-9]]", " : dẫn_chứng ", post)

    post = post.strip()
    if lowercase_opt:
      post = post.lower()
    tokens = tweet_tokenizer.tokenize(post.replace("’", "'").replace("…", "..."))
    
    post = " ".join(tokens)
    if use_segment:
        tokens = vncorenlp.tokenize(post.replace("’", "'").replace("…", "..."))
        tokens = [t for ts in tokens for t in ts]
    normPost = " ".join(tokens)

    if remove_punc_stopword:
      tokens = [t for t in normPost if not t in vnmese_stopwords]
    normPost = " ".join(tokens)

    normPost = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normPost)
    normPost = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normPost)
    normPost = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normPost)
    if use_segment:
        normPost = normPost.replace('< url >', '<url>')
        normPost = re.sub(r"# (\w+)", r'#\1', normPost)
    if truncation_method == "head_only":
      normPost = " ".join(normPost.split(" ")[:length])
    if truncation_method == "tail_only":
      normPost = " ".join(normPost.split(" ")[-length:])
    if truncation_method == "head_tail":
      normPost = " ".join(normPost.split(" ")[:int(length*0.25)]) + " " +  " ".join(normPost.split(" ")[-int(length*0.75):])
    return normPost

In [11]:
data_train = pd.read_csv(train_path)
data_val = pd.read_csv(val_path)
data_test = pd.read_csv(test_path)

In [12]:
#Time stamp feature
data_train['timestamp_post'] = pd.to_numeric(data_train['timestamp_post'], errors='coerce')
data_train['timestamp_post'] = data_train['timestamp_post'].astype('float')

data_test['timestamp_post'] = pd.to_numeric(data_test['timestamp_post'], errors='coerce')
data_test['timestamp_post'] = data_test['timestamp_post'].astype('float')

data_val['timestamp_post'] = pd.to_numeric(data_val['timestamp_post'], errors='coerce')
data_val['timestamp_post'] = data_val['timestamp_post'].astype('float')

#Number of shares feature
data_train['num_share_post'] = pd.to_numeric(data_train['num_share_post'], errors='coerce')
data_train['num_share_post'] = data_train['num_share_post'].astype('float')

data_test['num_share_post'] = pd.to_numeric(data_test['num_share_post'], errors='coerce')
data_test['num_share_post'] = data_test['num_share_post'].astype('float')

data_val['num_share_post'] = pd.to_numeric(data_val['num_share_post'], errors='coerce')
data_val['num_share_post'] = data_val['num_share_post'].astype('float')

#Number of likes feature
data_train['num_like_post'] = pd.to_numeric(data_train['num_like_post'], errors='coerce')
data_train['num_like_post'] = data_train['num_like_post'].astype('float')

data_test['num_like_post'] = pd.to_numeric(data_test['num_like_post'], errors='coerce')
data_test['num_like_post'] = data_test['num_like_post'].astype('float')

data_val['num_like_post'] = pd.to_numeric(data_val['num_like_post'], errors='coerce')
data_val['num_like_post'] = data_val['num_like_post'].astype('float')

#Number of comments feature
data_train['num_comment_post'] = pd.to_numeric(data_train['num_comment_post'], errors='coerce')
data_train['num_comment_post'] = data_train['num_comment_post'].astype('float')

data_test['num_comment_post'] = pd.to_numeric(data_test['num_comment_post'], errors='coerce')
data_test['num_comment_post'] = data_test['num_comment_post'].astype('float')

data_val['num_comment_post'] = pd.to_numeric(data_val['num_comment_post'], errors='coerce')
data_val['num_comment_post'] = data_val['num_comment_post'].astype('float')

data_train['post_message'] = data_train['post_message'].fillna('empty')
data_val['post_message'] = data_val['post_message'].fillna('empty')

In [13]:
def null_index_list(df, feature):
  return df[feature][df[feature].isnull()].index.tolist()

def fill_missing_median(df, feature):
  df[feature] = df[feature].fillna(df[feature].median())

In [14]:
nan_timestamp_idxs = null_index_list(data_train, "timestamp_post")
nan_like_idxs = null_index_list(data_train, "num_like_post")
nan_share_idxs = null_index_list(data_train, "num_share_post")
nan_comment_idxs = null_index_list(data_train, "num_comment_post")

val_nan_timestamp_idxs = null_index_list(data_val, "timestamp_post")
val_nan_like_idxs = null_index_list(data_val, "num_like_post")
val_nan_share_idxs = null_index_list(data_val, "num_share_post")
val_nan_comment_idxs = null_index_list(data_val, "num_comment_post")

test_nan_timestamp_idxs = null_index_list(data_test, "timestamp_post")
test_nan_like_idxs = null_index_list(data_test, "num_like_post")
test_nan_share_idxs = null_index_list(data_test, "num_share_post")
test_nan_comment_idxs = null_index_list(data_test, "num_comment_post")

In [15]:
tweet_tokenizer = TweetTokenizer()
rdrsegmenter = VnCoreNLP("/content/drive/MyDrive/VLSP-Fake-News-Detection/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg")

train_sents = []
test_sents = []
val_sents = []

# If PhoBERT: use_segment = True, otherwise False
# normalizePost(post, tweet_tokenizer, vncorenlp, use_segment=False, remove_punc_stopword=False, lowercase_opt=False, truncation_method="head_only", length=512)
for post in data_train.post_message:
  train_sents.append(normalizePost(post, tweet_tokenizer, rdrsegmenter, False, False, False, "head_only", 512))

for post in data_val.post_message:
  val_sents.append(normalizePost(post, tweet_tokenizer, rdrsegmenter, False, False, False, "head_only", 512))

for test_post in data_test.post_message:
  test_sents.append(normalizePost(test_post, tweet_tokenizer, rdrsegmenter, False, False, False, "head_only", 512))

In [16]:
from sentence_transformers import SentenceTransformer
sent_model = SentenceTransformer('bert-base-nli-mean-tokens')

train_sen_embeddings = sent_model.encode(train_sents)
test_sen_embeddings = sent_model.encode(test_sents)
val_sen_embeddings = sent_model.encode(val_sents)

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
train_pairwise_similarities = cosine_similarity(train_sen_embeddings)
val_pairwise_similarities = cosine_similarity(val_sen_embeddings)
test_pairwise_similarities = cosine_similarity(test_sen_embeddings)

In [18]:
#Fill missing value by copy their most similar post message's values to them
def fill_missing_similarity(df, pairwise_similarity, feature, features_list):
  for i in features_list:
    max_val, pos_i, pos_j = -1, -1, -1
    for j in range(len(pairwise_similarity[i])):
      if i == j or j in features_list:
        continue
      elif pairwise_similarity[i][j] > max_val:
        #max_val = pairwise_similarities[i][j]
        pos_i = i
        pos_j = j
    df[feature][pos_i] = df[feature][pos_j]

In [19]:
fill_missing_similarity(data_train, train_pairwise_similarities, "timestamp_post", nan_timestamp_idxs)
fill_missing_similarity(data_val, val_pairwise_similarities, "timestamp_post", val_nan_timestamp_idxs)
fill_missing_similarity(data_test, test_pairwise_similarities, "timestamp_post", test_nan_timestamp_idxs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [20]:
fill_missing_median(data_train, "num_like_post")
fill_missing_median(data_train, "num_share_post")
fill_missing_median(data_train, "num_comment_post")

fill_missing_median(data_val, "num_like_post")
fill_missing_median(data_val, "num_share_post")
fill_missing_median(data_val, "num_comment_post")

fill_missing_median(data_test, "num_like_post")
fill_missing_median(data_test, "num_share_post")
fill_missing_median(data_test, "num_comment_post")

In [21]:
def create_additional_features(df):
  df['year'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.year
  df['month'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.month
  df['day'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.day
  df['weekday'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.weekday
  df['hour'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.hour

In [22]:
create_additional_features(data_train)
create_additional_features(data_val)
create_additional_features(data_test)

In [23]:
data_train = data_train.drop(columns=["timestamp_post"])
data_val = data_val.drop(columns=["timestamp_post"])
data_test = data_test.drop(columns=["timestamp_post"])

In [24]:
train_datetime_statements = []
for (day, month, hour, weekday) in zip(data_train['day'], data_train['month'], data_train['hour'], data_train['weekday']):
  if weekday != 6:
    train_pattern_post = f"Bài viết được đăng vào thứ {weekday + 2}, ngày {day} tháng {month} lúc {hour} giờ"
  else:
    train_pattern_post = f"Bài viết được đăng vào Chúa nhật, ngày {day} tháng {month} lúc {hour} giờ"
  train_datetime_statements.append(train_pattern_post)

print(train_datetime_statements)

['Bài viết được đăng vào thứ 6, ngày 3 tháng 4 lúc 20 giờ', 'Bài viết được đăng vào thứ 6, ngày 8 tháng 5 lúc 11 giờ', 'Bài viết được đăng vào thứ 7, ngày 6 tháng 6 lúc 1 giờ', 'Bài viết được đăng vào thứ 7, ngày 13 tháng 6 lúc 4 giờ', 'Bài viết được đăng vào thứ 2, ngày 9 tháng 3 lúc 7 giờ', 'Bài viết được đăng vào thứ 3, ngày 26 tháng 5 lúc 4 giờ', 'Bài viết được đăng vào thứ 4, ngày 13 tháng 5 lúc 19 giờ', 'Bài viết được đăng vào thứ 5, ngày 28 tháng 5 lúc 1 giờ', 'Bài viết được đăng vào thứ 4, ngày 15 tháng 4 lúc 11 giờ', 'Bài viết được đăng vào thứ 4, ngày 20 tháng 5 lúc 7 giờ', 'Bài viết được đăng vào thứ 5, ngày 21 tháng 5 lúc 5 giờ', 'Bài viết được đăng vào thứ 3, ngày 31 tháng 3 lúc 11 giờ', 'Bài viết được đăng vào thứ 2, ngày 13 tháng 4 lúc 1 giờ', 'Bài viết được đăng vào thứ 5, ngày 20 tháng 2 lúc 11 giờ', 'Bài viết được đăng vào thứ 6, ngày 26 tháng 6 lúc 8 giờ', 'Bài viết được đăng vào thứ 3, ngày 28 tháng 4 lúc 5 giờ', 'Bài viết được đăng vào thứ 4, ngày 29 tháng 4 lúc 0 

In [25]:
val_datetime_statements = []
for (day, month, hour, weekday) in zip(data_val['day'], data_val['month'], data_val['hour'], data_val['weekday']):
  if weekday != 6:
    val_pattern_post = f"Bài viết được đăng vào thứ {weekday + 2}, ngày {day} tháng {month} lúc {hour} giờ"
  else:
    val_pattern_post = f"Bài viết được đăng vào Chúa nhật, ngày {day} tháng {month} lúc {hour} giờ"
  val_datetime_statements.append(val_pattern_post)

In [26]:
test_datetime_statements = []
for (day, month, hour, weekday) in zip(data_test['day'], data_test['month'], data_test['hour'], data_test['weekday']):
  if weekday != 6:
    test_pattern_post = f"Bài viết được đăng vào thứ {weekday + 2}, ngày {day} tháng {month} lúc {hour} giờ"
  else:
    test_pattern_post = f"Bài viết được đăng vào Chúa nhật, ngày {day} tháng {month} lúc {hour} giờ"
  test_datetime_statements.append(test_pattern_post)

print(train_datetime_statements)

['Bài viết được đăng vào thứ 6, ngày 3 tháng 4 lúc 20 giờ', 'Bài viết được đăng vào thứ 6, ngày 8 tháng 5 lúc 11 giờ', 'Bài viết được đăng vào thứ 7, ngày 6 tháng 6 lúc 1 giờ', 'Bài viết được đăng vào thứ 7, ngày 13 tháng 6 lúc 4 giờ', 'Bài viết được đăng vào thứ 2, ngày 9 tháng 3 lúc 7 giờ', 'Bài viết được đăng vào thứ 3, ngày 26 tháng 5 lúc 4 giờ', 'Bài viết được đăng vào thứ 4, ngày 13 tháng 5 lúc 19 giờ', 'Bài viết được đăng vào thứ 5, ngày 28 tháng 5 lúc 1 giờ', 'Bài viết được đăng vào thứ 4, ngày 15 tháng 4 lúc 11 giờ', 'Bài viết được đăng vào thứ 4, ngày 20 tháng 5 lúc 7 giờ', 'Bài viết được đăng vào thứ 5, ngày 21 tháng 5 lúc 5 giờ', 'Bài viết được đăng vào thứ 3, ngày 31 tháng 3 lúc 11 giờ', 'Bài viết được đăng vào thứ 2, ngày 13 tháng 4 lúc 1 giờ', 'Bài viết được đăng vào thứ 5, ngày 20 tháng 2 lúc 11 giờ', 'Bài viết được đăng vào thứ 6, ngày 26 tháng 6 lúc 8 giờ', 'Bài viết được đăng vào thứ 3, ngày 28 tháng 4 lúc 5 giờ', 'Bài viết được đăng vào thứ 4, ngày 29 tháng 4 lúc 0 

In [27]:
train_num_statements = []
for (num_like, num_share, num_comment) in zip(data_train['num_like_post'], data_train['num_share_post'], data_train['num_comment_post']):
  train_num_post = f"Bài viết có {num_like} lượt like, {num_share} lượt chia sẻ và {num_comment} bình luận"
  train_num_statements.append(train_num_post)

In [28]:
val_num_statements = []
for (num_like, num_share, num_comment) in zip(data_val['num_like_post'], data_val['num_share_post'], data_val['num_comment_post']):
  val_num_post = f"Bài viết có {num_like} lượt like, {num_share} lượt chia sẻ và {num_comment} bình luận"
  val_num_statements.append(val_num_post)

In [29]:
test_num_statements = []
for (num_like, num_share, num_comment) in zip(data_test['num_like_post'], data_test['num_share_post'], data_test['num_comment_post']):
  test_num_post = f"Bài viết có {num_like} lượt like, {num_share} lượt chia sẻ và {num_comment} bình luận"
  test_num_statements.append(test_num_post)

In [30]:
train_modified_texts = []
for (train_num_post, train_datetime_post, tr_sent) in zip(train_num_statements, train_datetime_statements, train_sents):
  train_text = train_num_post + ". " + train_datetime_post + ". " + tr_sent
  train_modified_texts.append(train_text)

In [31]:
val_modified_texts = []
for (val_num_post, val_datetime_post, val_sent) in zip(val_num_statements, val_datetime_statements, val_sents):
  val_text = val_num_post + ". " + val_datetime_post  + ". " + val_sent
  val_modified_texts.append(val_text)

In [32]:
test_modified_texts = []
for (test_num_post, test_datetime_post, test_sent) in zip(test_num_statements, test_datetime_statements, test_sents):
  test_text = test_num_post + ". " + test_datetime_post + ". " + test_sent
  test_modified_texts.append(test_text)

In [60]:
def convert_tokens_to_ids(text, tokenizer, max_seq_length=256, labels=None):
    inputs = tokenizer.encode_plus(text, padding='max_length', max_length=max_seq_length, truncation=True)
    input_ids = inputs['input_ids']
    attention_masks = inputs['attention_mask']

    if labels is not None:
        return torch.tensor(input_ids, dtype=torch.long), torch.tensor(attention_masks, dtype=torch.long), torch.tensor(labels, dtype=torch.long)
    return torch.tensor(input_ids, dtype=torch.long), torch.tensor(attention_masks, dtype=torch.long)

In [34]:
if torch.cuda.is_available():
  device = torch.device('cuda')
  print(torch.cuda.get_device_name())
else:
  device = torch.device('cpu')

Tesla P100-PCIE-16GB


In [45]:
test_config_path = "/content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/electra/model_7370.bin"
test_config = ElectraConfig.from_pretrained("/content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/electra", num_labels=1, output_hidden_states=True)
test_model = ElectraReINTELClassification.from_pretrained(test_config_path, config=test_config)
test_model.to(device)
test_tokenizer = ElectraTokenizer.from_pretrained("/content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/electra", do_lower_case=False)

In [53]:
sample_sent = test_modified_texts[0]
print(sample_sent)

Bài viết có 41.0 lượt like, 7.0 lượt chia sẻ và 3.0 bình luận. Bài viết được đăng vào thứ 6, ngày 22 tháng 5 lúc 15 giờ. Các lời khai cố tình bị rút ra để áp án tử cho Hải đã được thu thập . Sắp kết thúc về vụ án ép tội chết lớn của lịch sử tư pháp .


In [61]:
sample_ids, sample_masks = convert_tokens_to_ids(sample_sent, test_tokenizer)
print(sample_ids)
print(sample_masks)

tensor([   2, 2085, 1097,  422, 2881,  398,  377, 1345, 5910,  397,  391,  398,
         377, 1345,  834, 1016,  420,  383,  398,  377,  730, 1075,  398, 2085,
        1097,  424, 1086,  442,  620,  389,  397,  465, 1262,  519,  387,  756,
         959,  652,  398,  709,  744,  789,  938,  523,  451, 1374,  438,  436,
         999,  601,  852,  428,  895,  432,  424,  658, 1853,  398,  345, 3858,
         549, 1181,  443,  537,  601, 1835, 1098, 1013,  557,  421,  867,  595,
         563,  685,  398,    3,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   

In [62]:
sample_dataset = TensorDataset(sample_ids, sample_masks)
sample_dataloader = DataLoader(sample_dataset, batch_size=32, shuffle=False)

In [71]:
for i, (input_ids, masks) in enumerate(sample_dataloader):
  if i % 20 == 0 or i == len(sample_dataloader):
      print(f"Predicted {i} posts.")
  y_pred = test_model(input_ids.cuda(), attention_mask=masks.cuda())
  y_pred = y_pred.squeeze().detach().cpu().numpy()
  sample_preds = np.atleast_1d(y_pred) if sample_preds is None else np.concatenate([sample_preds, np.atleast_1d(y_pred)])

sample_preds = sigmoid(sample_preds)
print(sample_preds)

Predicted 0 posts.


ValueError: ignored